In [ ]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))


import csv
from datetime import datetime
from datetime import timedelta
import os
import gc
from numpy import abs, max 
import scipy
import numpy as np
from matplotlib import pyplot as plt
import sys
import pandas as pd

import random
import time

from sklearn import preprocessing

from tqdm import tqdm

In [ ]:
subject = 'A'

In [ ]:
len_data = np.zeros(0)
idx = np.zeros(0)

root_path = '../data/raw_data/'+subjects+'/'
files = os.listdir(root_path)
num_file = int(files[-1][:-4])

for i in range(num_file-1):
    npz = np.load(root_path+str(i)+'.npz')
    acc = npz['d']
    cnt_pushup = npz['p']
    cnt_squat = npz['s']
    cnt_abs = npz['a']

    for j in range(i+1,num_file):
        npz2 = np.load(root_path+str(j)+'.npz')
        acc2 = npz2['d']
        cnt_pushup2 = npz2['p']
        cnt_squat2 = npz2['s']
        cnt_abs2 = npz2['a']

        if acc.shape[1] + acc2.shape[1] <= 27000:
            plus_data = np.append(acc,acc2,axis=1)
            plus_cnt_p = cnt_pushup+cnt_pushup2
            plus_cnt_s = cnt_squat+cnt_squat2
            plus_cnt_a = cnt_abs+cnt_abs2


            np.savez('../data/plus/'+subject+'/'+str(i)+'+'+str(j)+'.npz',d = plus_data, p = plus_cnt_p, s = plus_cnt_s, a = plus_cnt_a)

In [ ]:
window_size = 5
num_aug = 50

for i in range(num_file-1):
    for j in range(i+1,num_file):
        filepath = '../data/plus/'+subject+'/'+str(i)+'+'+str(j)+'.npz' 
        
        if os.path.exists(filepath) == True: 
            
            dataset = np.zeros((num_aug,3,35000))
            cnt_p = np.zeros(0)
            cnt_s = np.zeros(0)
            cnt_a = np.zeros(0)
            
            
            new_dir_path = '../data/augmented_data/'+subject+'/plus/'+str(i)+'+'+str(j)+'/'
            if os.path.exists(new_dir_path) == False:
                os.mkdir(new_dir_path)
            
            npz = np.load(filepath)
            acc = npz['d']
            cnt_pushup = npz['p']
            cnt_squat = npz['s']
            cnt_abs = npz['a']
            
            
            
            STD = np.zeros((3,int(len(acc[0])/window_size)))
            for d in range(3):
                k = 0
                for idx in np.arange(0,len(acc[0]),window_size):
                    window = acc[d][idx:idx+window_size]
                    std = np.std(window)
                    STD[d][k] = std
                    k = k + 1
            
            Multi_STD = STD[0]+STD[1]+STD[2] 
            
            idx_std = np.where((Multi_STD <= 0.1)) 

            sep_idx = np.zeros(1) 
            m = 0
            
            for n in idx_std[0]:
                if n >= sep_idx[m] + 200:
                    sep_idx = np.append(sep_idx,n)
                    m = m + 1
            cut_idx = sep_idx*window_size 


            cut_data = [] 
            
            
            for l in range(len(cut_idx)-1):
                cut_data.append(acc[:,int(cut_idx[l]):int(cut_idx[l+1])])
            cut_data.append(acc[:,int(cut_idx[len(cut_idx)-1]):])

            
            for t in range(num_aug): 
                shuffl_data = random.sample(cut_data,len(cut_data))
                stretched_data = []
                augmented_data = np.zeros((3,0)) 
                for k in range(len(shuffl_data)): 
                    wide_r = random.uniform(0.7,1.3)
                    height_r = random.uniform(0.6,1.4) 
                    shift_r = random.uniform(-5,5)

                    a = shuffl_data[k] 
                    a_stretch_0 = np.interp(np.arange(0,len(a[0]),wide_r),np.arange(0,len(a[0])),a[0])*height_r
                    a_stretch_1 = np.interp(np.arange(0,len(a[1]),wide_r),np.arange(0,len(a[1])),a[1])*height_r
                    a_stretch_2 = np.interp(np.arange(0,len(a[2]),wide_r),np.arange(0,len(a[2])),a[2])*height_r
                      
                    a_stretch = np.vstack((a_stretch_0,np.vstack((a_stretch_1,a_stretch_2))))



                    stretched_data.append(a_stretch)

                for l in range(len(stretched_data)):
                    augmented_data = np.append(augmented_data,stretched_data[l],axis=1)

                for s in range(3):
                    shift_r = random.uniform(-5,5)
                    augmented_data[s] = augmented_data[s] + shift_r

                Data = np.zeros((3,35000))

                Data[:,:augmented_data.shape[1]] = augmented_data

                dataset[t] = Data
                
                cnt_p = np.append(cnt_p,cnt_pushup)
                cnt_s = np.append(cnt_s,cnt_squat)
                cnt_a = np.append(cnt_a,cnt_abs)
                
            np.savez(new_dir_path+str(num_aug)+'.npz',d = dataset, p = cnt_p, s = cnt_s, a = cnt_a)